In [1]:
import os
import pathlib

import dotenv
import numpy as np
import xarray as xr
import xroms

from typing import Literal

# Filepaths

In the project code directory there is a `.env` file that defines various filepaths. For example, one of the lines is:
- `SANTA_ROSA_RIDGE_MODEL_OUTPUT=${GROUP_SCRATCH}/santa_rosa_ridge_roms`.

We can load the values from this `.env` file using the `dotenv` package. Passing `interpolate=True` tells `dotenv_values` to interpolate the value of environment variables such as `GROUP_SCRATCH` so that we get the full path.

Then we use `pathlib` to extract all the ROMS history files and the grid file.

In [2]:
# get paths
ENV_VARIABLES = dotenv.dotenv_values(interpolate=True)
MODEL_OUTPUT_DIR = ENV_VARIABLES['SANTA_ROSA_RIDGE_MODEL_OUTPUT']
MODEL_OUTPUT_DIR

'/scratch/groups/leift/santa_rosa_ridge_roms'

In [3]:
# get roms filepaths
HIS_FILES = sorted(pathlib.Path(MODEL_OUTPUT_DIR).glob("*_his.*.nc"))
GRID_FILE = next(pathlib.Path(MODEL_OUTPUT_DIR).glob("*_grd.nc"), None)
print(f"Found {len(HIS_FILES)} history files and {'a' if GRID_FILE is not None else 'no'} grid file.")

Found 59 history files and a grid file.


# Opening the dataset

## The time dimension

Different versions of ROMS use different conventions for the time dimension. Sometimes this means we need to do a little work before we begin our analysis. The dataset we're opening here has a `time` dimension and defines an `ocean_time` variable that stores the time in seconds since initialisation. This causes a couple of issues:

1) The analysis software we use, `xroms`, expects an `ocean_time` dimension.
2) In the datasets, the `time` dimension has no coordinate attached to it and consequently we cannot immediately concatenate the output files along the time dimension.

Fortunately, these issues can be solved with a simple one line function. We then open the files as a multi-file dataset using `xr.open_mfdataset`, passing the preprocessing function and the keyword arguments suggested by the [`xroms` documentation](https://xroms.readthedocs.io/en/latest/io.html#open-mfdataset).

In [4]:
def make_ocean_time_dim(ds: xr.Dataset) -> xr.Dataset:
    """Make ocean time the dimension."""
    return ds.swap_dims(time="ocean_time")

# open his files as one dataset
ds = xr.open_mfdataset(HIS_FILES, 
                       preprocess=make_ocean_time_dim, 
                       join="outer", 
                       compat="override", 
                       combine="by_coords", 
                       data_vars="minimal", 
                       coords="minimal"
                      )
ds


<xarray.Dataset> Size: 4TB
Dimensions:     (ocean_time: 268, auxil: 6, eta_rho: 1602, xi_rho: 1602,
                 xi_u: 1601, eta_v: 1601, s_rho: 200, s_w: 201)
Coordinates:
  * ocean_time  (ocean_time) float64 2kB 5.835e+08 5.836e+08 ... 5.847e+08
Dimensions without coordinates: auxil, eta_rho, xi_rho, xi_u, eta_v, s_rho, s_w
Data variables: (12/14)
    time_step   (ocean_time, auxil) float64 13kB dask.array<chunksize=(1, 6), meta=np.ndarray>
    zeta        (ocean_time, eta_rho, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    ubar        (ocean_time, eta_rho, xi_u) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    vbar        (ocean_time, eta_v, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    u           (ocean_time, s_rho, eta_rho, xi_u) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    v           (ocean_time, s_rho, eta_v, xi_rho) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    ...          ...
    passive     (ocean_time, s_rho, eta_rho, xi_rho) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    rho         (ocean_time, s_rho, eta_rho, xi_rho) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    Akv         (ocean_time, s_w, eta_rho, xi_rho) float32 553GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    hbls        (ocean_time, eta_rho, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    hbbl        (ocean_time, eta_rho, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    w           (ocean_time, s_rho, eta_rho, xi_rho) float32 550GB dask.array<chunksize=(1, 34, 321, 321), meta=np.ndarray>
Attributes: (12/42)
    title:              Santa Rosa Ridge (62 m)
    grid_file:           srridge_grd.nc
    forcing_files:      
    forcing_info:       
    init_file:           srridge_ini.20180629.nc
    ntimes:             180000
    ...                 ...
    tide_options:       OFF
    river_frc_options:  OFF
    pipe_frc_options:   OFF
    particle_options:   full seed , constant density , concentration(m^-3):  ...
    git_version:        1d85d1445d7ea2f8da0388df1e5340666804af36
    type:               ROMS history file

## Dask and chunking

We have now opened the dataset. Note that all the Data variables store their data as `dask.array<chunksize=...`. 

This is a key aspect of working with `ROMS` output using `xarray` and `xroms`. `Dask` is a library that facilitates lazy computation --- i.e. deferring computation until it's actually required. Importantly, that also includes the loading of data from disk into memory. This enables us to work with the entire dataset even though it's many times larger than the amount of memory we have available. 

Effectively utilising chunking is one of the trickier parts of working with `Dask` but has a huge effect on performance. A chunk is the smallest unit of data that `Dask` will work with. For example, if we were to load the first datapoint of salinity 
```python
first_salt = ds.salt.isel(ocean_time=0, s_rho=0, eta_rho=0, xi_rho=0).load()
```
then we actually load the entire chunk containing that datapoint. Note here the ``.load()`` method call. Data selection is a lazy computation but we can force `Dask` to perform the compuation using the ``.load()`` or ``.compute()`` method. 

Some important considerations when working with `Dask` and chunking:
- The order of computations is critical.
    - A good rule of thumb is to perform any computation that reduces the size of your dataset as soon as possible.
- Certain computations require a particular chunking layout.
    - For example, when interpolating `xroms` generally requires the entire interpolation dimension to be one chunk.
- You can specify the chunks when loading the dataset.
    - Pass a dictionary as the ``chunks`` keyword argument, e.g. ``chunks={"ocean_time": 1, "s_rho": -1, "s_w": -1}``, to ```xr.open_mfdataset```.
    - Keys are dimension names.
    - Values are the chunk size --- the special value -1 makes the whole dimension one chunk.
- It is possible to rechunk the dataset.
    - This can be an expensive operation.
    - Often a good tactic when manipulating the data in space and time is to first load the data with an ``ocean_time`` chunksize of 1 and then rechunk after performing the spatial computations.
  
## The grid
To perform computations we, and `xroms`, require various grid quantities. Some versions of ROMS save these in the output files but here we have to add them explicitly. This also requires a bit of work but is fairly straight-forward. First, we need the variables that define the vertical grid. For an explanation of the how the vertical coordinate system works check out the [Roms wiki](https://www.myroms.org/wiki/Vertical_S-coordinate).

1) First, we create the `s-coordinates` which are uniformly spaced between -1 and 0.
2) Then we extract the grid stretching parameters which here are saved as attributes of the output files.

Now we add the horizontal grid quantities saved in the grid file. Also, `xroms` expects certain grid quantities, i.e. the latitudes and longitudes, to be coordinates.

3) We open the grid file and make the lon and lat variables coordinates.
4) Finally, we merge the two datasets.



In [5]:
def add_s_coordinates(ds: xr.Dataset) -> xr.Dataset:
    """Construct and add the s-coordinates."""
    ds["s_w"] = (s_w := np.linspace(-1,0,ds.s_w.size, endpoint=True))
    ds["s_rho"] = s_w[:-1] + np.diff(s_w) / 2
    return ds

def add_grid_stretching(ds: xr.Dataset, Vtransform: Literal[1,2] = 2) -> xr.Dataset:
    """Extract the grid stretching out of attrs and add to dataset."""
    ds["Cs_r"] = ("s_rho", ds.attrs["Cs_r"])
    ds["Cs_w"] = ("s_w", ds.attrs["Cs_w"])
    ds["Vtransform"] = Vtransform
    return ds

ds = add_s_coordinates(ds)
ds = add_grid_stretching(ds)
grd = xr.open_dataset(GRID_FILE)
grd = grd.set_coords(["lon_rho", "lat_rho", "lon_coarse", "lat_coarse"])
ds = ds.merge(grd)
ds

<xarray.Dataset> Size: 4TB
Dimensions:       (ocean_time: 268, auxil: 6, eta_rho: 1602, xi_rho: 1602,
                   xi_u: 1601, eta_v: 1601, s_rho: 200, s_w: 201, one: 1,
                   eta_coarse: 802, xi_coarse: 802)
Coordinates:
  * ocean_time    (ocean_time) float64 2kB 5.835e+08 5.836e+08 ... 5.847e+08
  * s_w           (s_w) float64 2kB -1.0 -0.995 -0.99 ... -0.01 -0.005 0.0
  * s_rho         (s_rho) float64 2kB -0.9975 -0.9925 ... -0.0075 -0.0025
    lon_rho       (eta_rho, xi_rho) float64 21MB ...
    lat_rho       (eta_rho, xi_rho) float64 21MB ...
    lon_coarse    (eta_coarse, xi_coarse) float64 5MB ...
    lat_coarse    (eta_coarse, xi_coarse) float64 5MB ...
Dimensions without coordinates: auxil, eta_rho, xi_rho, xi_u, eta_v, one,
                                eta_coarse, xi_coarse
Data variables: (12/32)
    time_step     (ocean_time, auxil) float64 13kB dask.array<chunksize=(1, 6), meta=np.ndarray>
    zeta          (ocean_time, eta_rho, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    ubar          (ocean_time, eta_rho, xi_u) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    vbar          (ocean_time, eta_v, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    u             (ocean_time, s_rho, eta_rho, xi_u) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    v             (ocean_time, s_rho, eta_v, xi_rho) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    ...            ...
    tra_lat       (one) float64 8B ...
    rotate        (one) float64 8B ...
    xy_flip       (one) float64 8B ...
    angle_coarse  (eta_coarse, xi_coarse) float64 5MB ...
    h_coarse      (eta_coarse, xi_coarse) float64 5MB ...
    mask_coarse   (eta_coarse, xi_coarse) float64 5MB ...
Attributes: (12/42)
    title:              Santa Rosa Ridge (62 m)
    grid_file:           srridge_grd.nc
    forcing_files:      
    forcing_info:       
    init_file:           srridge_ini.20180629.nc
    ntimes:             180000
    ...                 ...
    tide_options:       OFF
    river_frc_options:  OFF
    pipe_frc_options:   OFF
    particle_options:   full seed , constant density , concentration(m^-3):  ...
    git_version:        1d85d1445d7ea2f8da0388df1e5340666804af36
    type:               ROMS history file

## XROMS

Finally we are ready to utilize `xroms`. The first thing to do is call [`xroms.roms_dataset`](https://xroms.readthedocs.io/en/latest/io.html#suggested-workflow-for-xroms) on our dataset. This does two things:
1) Add some additional variables to the dataset:
    -  The vertical coordinates `z`
    -  Various grid metrics e.g. the grid spacing
2) Calculates an `xgcm` grid object that we will need for horizontal interpolation.

In [6]:
ds, xgrid = xroms.roms_dataset(ds)
ds

<xarray.Dataset> Size: 22TB
Dimensions:       (ocean_time: 268, auxil: 6, eta_rho: 1602, xi_rho: 1602,
                   xi_u: 1601, eta_v: 1601, s_rho: 200, s_w: 201, one: 1,
                   eta_coarse: 802, xi_coarse: 802)
Coordinates: (12/21)
  * ocean_time    (ocean_time) float64 2kB 5.835e+08 5.836e+08 ... 5.847e+08
  * s_w           (s_w) float64 2kB -1.0 -0.995 -0.99 ... -0.01 -0.005 0.0
  * s_rho         (s_rho) float64 2kB -0.9975 -0.9925 ... -0.0075 -0.0025
    lon_rho       (eta_rho, xi_rho) float64 21MB 239.4 239.4 ... 240.4 240.4
    lat_rho       (eta_rho, xi_rho) float64 21MB 33.15 33.15 ... 34.05 34.05
    lon_coarse    (eta_coarse, xi_coarse) float64 5MB ...
    ...            ...
    z_w_v         (ocean_time, s_w, eta_v, xi_rho) float64 1TB dask.array<chunksize=(1, 201, 801, 801), meta=np.ndarray>
    z_w_psi       (ocean_time, s_w, eta_v, xi_u) float64 1TB dask.array<chunksize=(1, 201, 801, 801), meta=np.ndarray>
    z_rho         (ocean_time, s_rho, eta_rho, xi_rho) float64 1TB dask.array<chunksize=(1, 200, 801, 801), meta=np.ndarray>
    z_rho_u       (ocean_time, s_rho, eta_rho, xi_u) float64 1TB dask.array<chunksize=(1, 200, 801, 801), meta=np.ndarray>
    z_rho_v       (ocean_time, s_rho, eta_v, xi_rho) float64 1TB dask.array<chunksize=(1, 200, 801, 801), meta=np.ndarray>
    z_rho_psi     (ocean_time, s_rho, eta_v, xi_u) float64 1TB dask.array<chunksize=(1, 200, 801, 801), meta=np.ndarray>
Dimensions without coordinates: auxil, one
Data variables: (12/51)
    time_step     (ocean_time, auxil) float64 13kB dask.array<chunksize=(1, 6), meta=np.ndarray>
    zeta          (ocean_time, eta_rho, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    ubar          (ocean_time, eta_rho, xi_u) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    vbar          (ocean_time, eta_v, xi_rho) float32 3GB dask.array<chunksize=(1, 801, 801), meta=np.ndarray>
    u             (ocean_time, s_rho, eta_rho, xi_u) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    v             (ocean_time, s_rho, eta_v, xi_rho) float32 550GB dask.array<chunksize=(1, 23, 201, 201), meta=np.ndarray>
    ...            ...
    dz_v          (ocean_time, s_rho, eta_v, xi_rho) float64 1TB dask.array<chunksize=(1, 200, 801, 801), meta=np.ndarray>
    dz_w_v        (ocean_time, s_w, eta_v, xi_rho) float64 1TB dask.array<chunksize=(1, 201, 801, 801), meta=np.ndarray>
    dz_psi        (ocean_time, s_rho, eta_v, xi_u) float64 1TB dask.array<chunksize=(1, 200, 801, 801), meta=np.ndarray>
    dz_w_psi      (ocean_time, s_w, eta_v, xi_u) float64 1TB dask.array<chunksize=(1, 201, 801, 801), meta=np.ndarray>
    dA            (eta_rho, xi_rho) float64 21MB 3.906e+03 ... 3.906e+03
    rho0          int64 8B 1025
Attributes: (12/42)
    title:              Santa Rosa Ridge (62 m)
    grid_file:           srridge_grd.nc
    forcing_files:      
    forcing_info:       
    init_file:           srridge_ini.20180629.nc
    ntimes:             180000
    ...                 ...
    tide_options:       OFF
    river_frc_options:  OFF
    pipe_frc_options:   OFF
    particle_options:   full seed , constant density , concentration(m^-3):  ...
    git_version:        1d85d1445d7ea2f8da0388df1e5340666804af36
    type:               ROMS history file